In [1]:
import pandas as pd
import numpy as np
import re

In [2]:
#get tickers and crsp db characteristics

### GET DAILY STOCK CHARACTERISTICS, NOT MONTHLY ###

stock = pd.read_csv('crsp_stock.csv', parse_dates=['date'])
print(stock.info())
stock.head(5)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 923401 entries, 0 to 923400
Data columns (total 11 columns):
PERMNO      923401 non-null int64
date        923401 non-null datetime64[ns]
SHRCD       918188 non-null float64
TICKER      908421 non-null object
PRIMEXCH    918188 non-null object
CUSIP       923401 non-null object
PRC         902949 non-null float64
VOL         907698 non-null float64
RET         912718 non-null object
SHROUT      917416 non-null float64
SPREAD      24509 non-null float64
dtypes: datetime64[ns](1), float64(5), int64(1), object(4)
memory usage: 77.5+ MB
None


,PERMNO,date,SHRCD,TICKER,PRIMEXCH,CUSIP,PRC,VOL,RET,SHROUT,SPREAD
0,10001,2006-01-31,11.0,EWST,Q,36720410,9.490,757.0,-0.016495,2930.0,NaN
1,10001,2006-02-28,11.0,EWST,Q,36720410,9.390,439.0,-0.010537,2931.0,NaN
2,10001,2006-03-31,11.0,EWST,Q,36720410,10.990,777.0,0.170394,2932.0,NaN
3,10001,2006-04-28,11.0,EWST,Q,36720410,9.950,883.0,-0.094631,2932.0,NaN
4,10001,2006-05-31,11.0,EWST,Q,36720410,9.766,319.0,-0.010452,2932.0,NaN


In [3]:
###########################################

#nke2 = stock[stock['PERMNO'] == 57665]
#nke2[nke2['date'] == '2014-06-03']

In [4]:
#keep the following characteristics
    #share code = 10,11
    #primary exchange = N,A, or Q
    #absolute value price > 1
    #volume >= 0
    #returns >= -1, convert B and C to NaN values
    #shares outstanding > 0
    
    #drop missing observations

stock = stock.loc[(stock['SHRCD'] == 10.0 ) | (stock['SHRCD'] == 11.0)]
stock = stock.loc[(stock['PRIMEXCH'] == 'N') | (stock['PRIMEXCH'] == 'A') | (stock['PRIMEXCH'] == 'Q')]
stock['price'] = stock['PRC'].abs()
stock = stock.loc[(stock['price'] >= 1)]
stock = stock.loc[(stock['VOL'] >= 0)]
stock['RET'] = stock['RET'].replace(['B','C'], np.nan).apply(float)
stock = stock.loc[(stock['RET'] >= -1)]
stock = stock.loc[(stock['SHROUT'] > 0)]

stock['MCAP'] = stock['SHROUT'] * stock['price']

stock.drop(columns = ['CUSIP','PRC','SPREAD','SHRCD','PRIMEXCH','SHROUT'], inplace = True)
stock = stock.dropna()

stock.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 509559 entries, 0 to 923400
Data columns (total 7 columns):
PERMNO    509559 non-null int64
date      509559 non-null datetime64[ns]
TICKER    509559 non-null object
VOL       509559 non-null float64
RET       509559 non-null float64
price     509559 non-null float64
MCAP      509559 non-null float64
dtypes: datetime64[ns](1), float64(4), int64(1), object(1)
memory usage: 31.1+ MB


In [5]:
#import twitter data from stata file

tweets = pd.read_stata('Tweets_C2.dta')
tweets.info(null_counts=True)

<class 'pandas.core.frame.DataFrame'>
Int64Index: 3439754 entries, 0 to 3439753
Data columns (total 7 columns):
twhandle         3439754 non-null object
permno           3439754 non-null int32
tweet_content    3439754 non-null object
timestr          3439754 non-null object
date             3439754 non-null datetime64[ns]
time             3439754 non-null object
tweet_id         3439754 non-null float32
dtypes: datetime64[ns](1), float32(1), int32(1), object(4)
memory usage: 183.7+ MB


In [6]:
tweets.sample(10)

,twhandle,permno,tweet_content,timestr,date,time,tweet_id
264534,BEBE_STORES,86150,Loves it! xo RT @VivianGonzalez7: Today I'm we...,8:10:59 PM,2013-06-01,,264535.0
3261953,WESTERNUNION,91461,@lupstars No sabr?amos decirle informaci?n de ...,9:35:05 PM,2014-04-18,,3261954.0
2767242,STAPLES,75489,@windowsblog Good thing. 80% of tablet users s...,,2013-03-26,12:40 PM,2767243.0
2990046,TRIQUINTSEMI,80049,This video shows how our wide selection of rad...,7:03:39 PM,2013-11-01,,2990047.0
2020213,NEUSTAR,90763,.@YankeeGroup took a close look at the role nu...,4:20:01 PM,2014-03-11,,2020214.0
473938,CARMAX,89508,Our Midlothian store is ready for the CarMax T...,6:54:35 PM,2013-11-27,,473939.0
808582,DELTEK,92446,Increase in #UK agency reviews set to follow #...,2:00:21 PM,2012-08-20,,808583.0
675487,COCACOLA,11308,"@BernardDunne Great shot! Thanks for sharing, ...",,2009-12-07,8:09 AM,675488.0
577112,CHEESECAKE,77902,@brisaperstein that's so nice to hear! Thx :),,2011-10-16,7:06 PM,577113.0
974229,ELOQUA,13551,5 Ways to Bring Creativity Back to Your Cultur...,12:35:09 AM,2014-02-08,,974230.0


In [7]:
### https://arrow.readthedocs.io/en/latest/ ###

In [8]:
#identify PM, drop AM/PM from timestr and time, create an hour variable, and add 12 to hours of timestr and time

tweets['PM'] = tweets['timestr'].str.contains(r'\bPM\b',flags=re.IGNORECASE)
tweets.loc[(tweets['timestr'] == ''), ['PM']] = tweets['time'].str.contains(r'\bPM\b',flags=re.IGNORECASE)

tweets['timestr'] = tweets['timestr'].str.replace('PM','')
tweets['timestr'] = tweets['timestr'].str.replace('AM','')
tweets['time'] = tweets['time'].str.replace('PM','')
tweets['time'] = tweets['time'].str.replace('AM','')

In [9]:
tweets.loc[(tweets['time'] == ''), ['time']] = tweets['timestr']
tweets['hour'] = tweets['time'].str.extract(r'([0-9]+)')
tweets['minute'] = tweets['time'].str.extract(r'((?<=\:)[0-9]+)')

tweets[['hour','minute']] = tweets[['hour','minute']].apply(pd.to_numeric)

tweets.head(10)

,twhandle,permno,tweet_content,timestr,date,time,tweet_id,PM,hour,minute
0,1800FLOWERS,87162,@AWESOMEDAPHY I'm sorry for the disappointment...,2:52:28,2014-04-08,2:52:28,1.0,False,2,52
1,1800FLOWERS,87162,@ChitownFelice you should receive it shortly. ...,1:54:33,2014-05-06,1:54:33,2.0,True,1,54
2,1800FLOWERS,87162,@HamptonsMPiece Send me your phone number in a...,4:15:25,2014-05-17,4:15:25,3.0,True,4,15
3,1800FLOWERS,87162,@monsour I am sorry for the inconvenience. Plz...,7:58:28,2014-05-13,7:58:28,4.0,True,7,58
4,1800FLOWERS,87162,@RemarkablyAvg I am sry we havnt heard back fr...,10:57:02,2014-05-08,10:57:02,5.0,False,10,57
5,1800FLOWERS,87162,@shraddhaswaroop I'm very sorry to hear about ...,3:09:31,2014-05-09,3:09:31,6.0,False,3,9
6,1800FLOWERS,87162,@1800baskets @angiedepue Congrats. Enjoy that ...,12:53:17,2014-06-10,12:53:17,7.0,False,12,53
7,1800FLOWERS,87162,"@teleject Please DM us the order #, cust & re...",12:44:00,2014-02-23,12:44:00,8.0,True,12,44
8,1800FLOWERS,87162,@PrettySL I am so sorry. Pls follow & DM & I w...,11:41:00,2014-03-01,11:41:00,9.0,True,11,41
9,1800FLOWERS,87162,"@MissAlexBell I'm very sorry, but will be happ...",7:26:00,2014-03-01,7:26:00,10.0,True,7,26


In [10]:
###create 24 hour clock

tweets2 = tweets.copy(deep=True)
tweets2.loc[(tweets2['PM'] == True), ['hour']] = tweets2['hour'] + 11
tweets2['hour'] = tweets2['hour'].apply(str)
tweets2['minute'] = tweets2['minute'].apply(str)
tweets2['hour_24'] = pd.to_datetime(tweets2['hour'] + ':' + tweets2['minute'], format='%H:%M').dt.time

tweets3 = tweets2.drop(columns = ['twhandle','timestr','time','PM','hour','minute']).copy(deep=True)
tweets3.head()

,permno,tweet_content,date,tweet_id,hour_24
0,87162,@AWESOMEDAPHY I'm sorry for the disappointment...,2014-04-08,1.0,02:52:00
1,87162,@ChitownFelice you should receive it shortly. ...,2014-05-06,2.0,12:54:00
2,87162,@HamptonsMPiece Send me your phone number in a...,2014-05-17,3.0,15:15:00
3,87162,@monsour I am sorry for the inconvenience. Plz...,2014-05-13,4.0,18:58:00
4,87162,@RemarkablyAvg I am sry we havnt heard back fr...,2014-05-08,5.0,10:57:00


In [11]:
print(stock.info())
print(tweets3[['permno','date']].info())

<class 'pandas.core.frame.DataFrame'>
Int64Index: 509559 entries, 0 to 923400
Data columns (total 7 columns):
PERMNO    509559 non-null int64
date      509559 non-null datetime64[ns]
TICKER    509559 non-null object
VOL       509559 non-null float64
RET       509559 non-null float64
price     509559 non-null float64
MCAP      509559 non-null float64
dtypes: datetime64[ns](1), float64(4), int64(1), object(1)
memory usage: 31.1+ MB
None
<class 'pandas.core.frame.DataFrame'>
Int64Index: 3439754 entries, 0 to 3439753
Data columns (total 2 columns):
permno    int32
date      datetime64[ns]
dtypes: datetime64[ns](1), int32(1)
memory usage: 65.6 MB
None


In [12]:
###########################################

#tweets3['permno'].nunique()

1075

In [13]:
#############################################

#tweets3['permno'].value_counts()

57665    98239
89941    41163
47466    37020
75104    35624
92221    33425
10107    30350
91068    28777
11308    26438
87356    25339
47387    25271
92203    21171
55976    20119
12742    19643
13169    18915
89644    18522
81696    17769
90248    17217
47379    17203
65875    16462
12927    15814
93429    15806
28804    15729
57817    15507
83435    15348
27828    15101
77462    14896
84788    14550
12060    14207
85246    13874
12798    13739
         ...  
93124        1
76711        1
91350        1
79915        1
81292        1
10693        1
93174        1
72304        1
43553        1
13693        1
40440        1
42534        1
75527        1
39642        1
70578        1
84808        1
13139        1
93222        1
13109        1
11174        1
12449        1
75517        1
88319        1
86237        1
93420        1
56040        1
84281        1
13019        1
90125        1
10225        1
Name: permno, Length: 1075, dtype: int64

In [14]:
#################################################

#nke = tweets3[tweets3['permno'] == 57665]
#nke['date'].value_counts()

2014-06-10    784
2014-06-09    728
2014-06-03    714
2014-06-04    708
2014-06-02    699
2014-06-05    672
2014-06-06    538
2014-06-08    509
2014-06-07    470
2014-05-29    409
2014-03-06    363
2014-05-19    324
2014-05-20    313
2014-05-30    312
2014-03-26    298
2014-06-11    296
2014-06-01    294
2014-05-15    282
2014-05-27    281
2014-05-21    280
2014-05-16    279
2014-05-28    267
2014-05-31    258
2014-05-22    255
2012-07-21    252
2014-04-24    232
2014-05-23    228
2014-04-17    224
2012-07-15    216
2014-05-26    212
             ... 
2009-05-20      1
2008-12-08      1
2010-05-23      1
2009-03-05      1
2009-03-15      1
2009-10-24      1
2011-03-20      1
2009-03-14      1
2009-09-12      1
2008-05-08      1
2009-01-26      1
2009-03-03      1
2009-01-06      1
2008-11-12      1
2011-06-18      1
2008-05-15      1
2009-01-15      1
2009-01-21      1
2009-07-12      1
2009-08-24      1
2008-06-04      1
2008-04-27      1
2009-02-16      1
2011-08-13      1
2010-07-05

In [15]:
#######################################################

#nke2 = stock[stock['PERMNO'] == 57665]
#nke2[nke2['date'] == '2014-06-03']

,PERMNO,date,TICKER,VOL,RET,price,MCAP


In [16]:
tweets4 = pd.merge(tweets3, stock, left_on=['permno','date'], right_on=['PERMNO','date'], how = 'inner')
tweets4.drop(columns = ['permno'], inplace = True)
tweets4.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 125045 entries, 0 to 125044
Data columns (total 10 columns):
tweet_content    125045 non-null object
date             125045 non-null datetime64[ns]
tweet_id         125045 non-null float32
hour_24          125045 non-null object
PERMNO           125045 non-null int64
TICKER           125045 non-null object
VOL              125045 non-null float64
RET              125045 non-null float64
price            125045 non-null float64
MCAP             125045 non-null float64
dtypes: datetime64[ns](1), float32(1), float64(4), int64(1), object(3)
memory usage: 10.0+ MB


In [17]:
###################################################

#tweets4[tweets4['PERMNO'] == 57665]

,tweet_content,date,tweet_id,hour_24,PERMNO,TICKER,VOL,RET,price,MCAP
75006,"@victorialalala Sun's out, sneakers on, person...",2014-05-30,2029632.0,15:57:00,57665,NKE,567332.0,0.057574,76.91,53881684.71
75007,@L0V3B0MB Make it your second fastest mile nex...,2014-05-30,2029772.0,17:14:00,57665,NKE,567332.0,0.057574,76.91,53881684.71
75008,@simsirozge And conquer every challenge. #just...,2014-05-30,2029960.0,16:03:00,57665,NKE,567332.0,0.057574,76.91,53881684.71
75009,@VeronicaCaselli You know the distance. Now ge...,2014-05-30,2029997.0,17:15:00,57665,NKE,567332.0,0.057574,76.91,53881684.71
75010,@ritcheymd Keep kicking. #justdoit,2014-05-30,2030329.0,21:07:00,57665,NKE,567332.0,0.057574,76.91,53881684.71
75011,@Stef_Sanders Celebrate with a run. #justdoit,2014-05-30,2030483.0,16:48:00,57665,NKE,567332.0,0.057574,76.91,53881684.71
75012,@alisatobin Beat your morning time on the run ...,2014-05-30,2030654.0,13:38:00,57665,NKE,567332.0,0.057574,76.91,53881684.71
75013,@AmyBeyi Keep the streak going. #justdoit,2014-05-30,2030680.0,13:39:00,57665,NKE,567332.0,0.057574,76.91,53881684.71
75014,@CreativePixel20 Time to get back in the swing...,2014-05-30,2030795.0,20:03:00,57665,NKE,567332.0,0.057574,76.91,53881684.71
75015,@Amine_CarpeDiem Get back on your feet and go ...,2014-05-30,2030860.0,16:01:00,57665,NKE,567332.0,0.057574,76.91,53881684.71


In [18]:
tweets4['TICKER'].nunique()

984

In [19]:
#function to find all instances of cashtag tweets that also match company ticker

def find_ticker(text,ticker):
    x = '$'+ticker
    if x.lower() in text.lower():
        return True
    else:
        return False

In [20]:
#use find_ticker function to identify cashtag tweets about a company

tweets4['cashtag_tweet'] = tweets4.apply(lambda y: find_ticker(y['tweet_content'],y['TICKER']),axis=1)
tweets4['cashtag_tweet'].value_counts()

False    124627
True        418
Name: cashtag_tweet, dtype: int64

In [21]:
#identify financial tweets (excluding cashtags \$[a-zA-Z]+)

tweets4['fin_tweet'] = tweets4['tweet_content'].str.contains \
    (r'\bearnings?\b|\bconference calls?\b|\brevenue\b|\bquarterly\b|\bCEO\b|\bCFO\b|\bfinancial\b' \
     ,flags=re.IGNORECASE)
tweets4['fin_tweet'].value_counts()

False    123505
True       1540
Name: fin_tweet, dtype: int64

In [22]:
#identify tweets with URLs

tweets4['url_tweet'] = tweets4['tweet_content'].str.contains(r'http')
tweets4['url_tweet'].value_counts()

True     75392
False    49653
Name: url_tweet, dtype: int64

In [98]:
#combine cashtags and financial tweets into one variable
#combine cashtags, financial, and url tweets into one variable

tweets5 = tweets4.copy(deep=True)

tweets5['financial_tweet'] = (tweets5['fin_tweet'] | tweets5['cashtag_tweet']).astype(int)
tweets5['financial_url'] = (tweets5['financial_tweet'] & tweets5['url_tweet']).astype(int)
tweets5[['cashtag_tweet','fin_tweet','url_tweet']] = tweets5[['cashtag_tweet','fin_tweet','url_tweet']].astype(int)

print(tweets5['financial_tweet'].value_counts())
print(tweets5['financial_url'].value_counts())

tweets5.drop(columns=['url_tweet','tweet_id'], inplace=True)

tweets5.head()

0    123229
1      1816
Name: financial_tweet, dtype: int64
0    123599
1      1446
Name: financial_url, dtype: int64


,tweet_content,date,hour_24,PERMNO,TICKER,VOL,RET,price,MCAP,cashtag_tweet,fin_tweet,financial_tweet,financial_url
0,RT @Peggysue753: : @1800flowers: What I love m...,2014-04-30,12:42:00,87162,FLWS,41684.0,-0.031972,5.45,149242.8,0,0,0,0
1,What special memory with your Mom will you che...,2014-04-30,14:00:00,87162,FLWS,41684.0,-0.031972,5.45,149242.8,0,0,0,0
2,@mattsns78 I am very sorry for the trouble! I ...,2014-04-30,16:04:00,87162,FLWS,41684.0,-0.031972,5.45,149242.8,0,0,0,0
3,Congrats to today's #MyMom Twitter winner: @Ki...,2014-04-30,19:47:00,87162,FLWS,41684.0,-0.031972,5.45,149242.8,0,0,0,0
4,"RT @Tricia2272: @1800flowers A Rose bush, my M...",2014-04-30,12:42:00,87162,FLWS,41684.0,-0.031972,5.45,149242.8,0,0,0,0


In [100]:
#drop text 'RT' and drop '@user_ID' and drop '#$%^' symbols and drop http links

tweets5['tweet_content'] = tweets5['tweet_content'].str.lower()

tweets5['tweet_content'] = tweets5['tweet_content'].str.replace \
(r'(rt\s?|http\S+\s?|\#\s?|(\$[a-z0-9]+\s?)|\@[a-z0-9]+\s?|\:\s?|\.|\!|\?|\,)','').str.strip()

tweets5['tweet_content'][0]

'what i love most about mymom is her warm welcoming hugs mymom'

In [40]:
#ibes data from sas

ibes = pd.read_stata('ibes.dta')
ibes.drop(columns = ['ticker','statpers','measure','fpi'], inplace=True)
ibes.dropna(inplace=True)

In [26]:
#retain only the last observation per oftic and fpedats

earn = ibes.groupby(['oftic','fpedats'],as_index=False).last()
earn['eps_surprise'] = earn['actual'] - earn['meanest']

earn['eps_beat'] = np.where(earn['eps_surprise'] > 0, 1, 0)
earn.drop(columns = ['meanest','actual'], inplace=True)   
    
earn[earn['oftic'] == 'AAPL'].head(10)

,oftic,fpedats,numest,stdev,eps_surprise,eps_beat
399,AAPL,2004-03-31,16.0,0.00,0.0000,0
400,AAPL,2004-06-30,18.0,0.00,0.0021,1
401,AAPL,2004-09-30,21.0,0.00,0.0093,1
402,AAPL,2004-12-31,22.0,0.00,0.0200,1
403,AAPL,2005-03-31,22.0,0.00,0.0186,1
404,AAPL,2005-06-30,22.0,0.00,0.0129,1
405,AAPL,2005-09-30,23.0,0.00,0.0043,1
406,AAPL,2005-12-31,22.0,0.01,0.0229,1
407,AAPL,2006-03-31,22.0,0.01,0.0071,1
408,AAPL,2006-06-30,20.0,0.00,0.0171,1


In [ ]:
# combine earnings data with twitter/stock data to identify which dates are earnings dates

full_sample = pd.merge()

In [ ]:
#merge earnings data with twitter and stock data
    #calculate SUE
    #retain only periods surrounding earnings
    #check observation counts

    #determine if earnings is sufficient or if we should keep all tweets from stock/tweet file and predict day t+n returns
        #if this, consider impact from financial tweets vs. non-financial tweets?

In [ ]:
##########################################
#### OUTLINE FOR REMAINDER OF PROJECT ####
##########################################

#obtain tickers, stock characteristics, and other controls from vineets paper.
#must next pull in earnings dates and drop tweets that do not occur during an earnings period
#must calculate cumulative returns over given period

#preprocess the text data with n_grams, lematization, stopwords, etc... with scikit learn or gensim
#vectorize text data with 1. bag of words, 2. tdvif, 3. word2vec, 4. ask Jeff if anything else

#test various machine learning models on data to try and predict a probability of tweet being association with earnings beat
#consider deep learning approach as a more advanced method

#detemrine if we can use tweets that occur before an earnings date to predict a beat
#detremine if we can use tweets that occur before/on/just after earnings to predict PEAD
#consider what vineet has tested to see if we can improve those results

#consider if including a url in a financial tweet helps magnify the impact of the tweet on stock returns.

In [ ]:
#'((https?:\/\/)(\s)*(www\.)?|(www\.))(\s)*((\w|\s)+\.)*([\w\-\s]+\/)*([\w\-]+)((\?)?[\w\s]*=\s*[\w\%&]*)*'